In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation



Creacion del conjunto de datos.

Aqui vienen dos opciones:

1. Utilizar los "posts" simulados de un foro de videojuegos
2. Crear un archivo de texto llamado "textos.csv" con una columna "text" Esta columna se usaria como conjunto de datos.

In [ ]:
data = {
#     "post": [
#         "I love the new graphics on the latest game update",
#         "Has anyone experienced glitches with the new game patch?",
#         "What are your thoughts on the storyline of The Last Adventure?",
#         "The multiplayer mode is so competitive and exciting",
#         "I'm stuck on level 42, can anyone guide me?",
#         "The graphics are breathtaking in the new VR game",
#         "Experiencing lag in the online multiplayer sessions, any fixes?",
#         "The campaign mode of Warriors Quest is incredibly detailed",
#         "Anyone else excited about the upcoming expansion pack?",
#         "I miss the simplicity of older games, everything is so complex now"
#     ]
# }
        "post": [
        "¿Alguien más encuentra el jefe de la zona de Limgrave extremadamente difícil?",
        "Me encanta cómo Elden Ring integra la exploración en su vasto mundo. ¡No puedo dejar de explorar cada rincón!",
        "¿Cuál es la mejor estrategia para derrotar al dragón en el Monte Gelmir?",
        "Estoy teniendo problemas con el frame rate en PS5, ¿alguien sabe cómo solucionarlo?",
        "Acabo de obtener la espada de la noche y la llama, y es absolutamente devastadora.",
        "¿Alguien tiene consejos sobre cómo manejar las invasiones PvP? Siempre me encuentran desprevenido.",
        "Creo que encontré un bug que permite duplicar objetos, ¿debería reportarlo o aprovecharlo?",
        "¿Qué build recomiendan para un personaje enfocado en magia?",
        "El diseño de los enemigos es impresionante, especialmente los demiurgos menores.",
        "Estoy atascado en el Castillo Stormveil, ¿algún consejo para avanzar?",
        "La música en las áreas de la capital es increíble, realmente añade mucha atmósfera.",
        "¿Cuál es su arma favorita hasta ahora y por qué?",
        "He leído que jugar cooperativo hace que los jefes sean más fáciles, ¿alguien quiere unirse a mi juego?",
        "¿Cuántas horas les ha tomado llegar al final? Creo que voy muy lento.",
        "¿Alguien más siente que las mazmorras son algo repetitivas después de un tiempo?",
        "¡Finalmente derroté al jefe final después de 50 intentos! ¿Qué hago ahora?",
        "Me encantaría ver una expansión que explore más sobre la historia de los shardbearers."
    ]
}

#Descomentar alguna de las siguientes dos lineas para el ejemplo
df_posts = pd.DataFrame(data)
#df_posts = pd.read_csv('textos.csv', usecols=['text'])

df_posts.columns = ['post']


In [ ]:
df_posts.head()

In [ ]:
df_posts.count()
df_posts=df_posts[:500]
df_posts.count()

In [ ]:


# Descargar recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('punkt')

# Función para tokenizar y remover puntuación y stopwords
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Tokenizar
    tokens = nltk.word_tokenize(text)
    # Remover stopwords
    tokens = [word for word in tokens if word not in stopwords.words('spanish') and word.isalnum()]
    return ' '.join(tokens)

# Aplicar el preprocesamiento a las publicaciones
df_posts['processed'] = df_posts['post'].apply(preprocess_text)


#¿Qué son los temas que extrae LDA?

Los "temas" en LDA no son temas predefinidos o etiquetados como en una clasificación manual. Son simplemente agrupaciones de palabras que frecuentemente aparecen juntas en los documentos del conjunto de datos. Un "tema" podría consistir en palabras como `{reina, torre, peón, jaque mate}` que podríamos interpretar como relacionadas al ajedrez, pero el modelo solo ve este tema como un grupo de palabras que tienden a co-ocurrir.

Cada tema es una distribución de probabilidades sobre todas las palabras en el corpus, lo que significa que cada palabra tiene una probabilidad de pertenecer a un tema (o agrupamiento) específico.

In [ ]:
%%time
# Vectorizar el texto preprocesado
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(df_posts['processed'])

# Ajustar el modelo LDA
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(data_vectorized)

# Mostrar los temas
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Tema %d:" % (topic_idx+1))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

# Imprimir los temas
display_topics(lda_model, vectorizer.get_feature_names_out(), 5)
